<b>Import Libraries</b>


In [17]:
import pandas as pd
import numpy as np
from rdkit import Chem
import rdkit
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem, Draw
from rdkit.Chem import inchi

<b>Load Data</b>

In [18]:
df = pd.read_csv('data_train.csv')
df_test = pd.read_csv('smiles_test.csv')

In [19]:
print(df.head())

#task = bioassay
#bioassay definition: measurement of the concentration or potency of a substance by its effect on living cells or tissues.
#goal: prediction of active   
#1 (active)
#0 (unknown)
#-1 (inactive)

   Unnamed: 0                                             smiles  task1  \
0           0  CC1=NN(c2ccc(C)c(C)c2)C(=O)C1N=Nc1cccc(-c2cccc...      0   
1           1                     CCOP(=S)(OCC)Oc1ccc(S(C)=O)cc1      0   
2           2  CN(c1ccccc1CN=c1[nH]c(Nc2ccc3c(c2)CC(=O)N3)ncc...      0   
3           3  CC#CC1(O)CCC2C3CCC4=CC(=O)CCC4=C3C(c3ccc(N(C)C...      0   
4           4                  O=C(O)C1(C(O)=Nc2ccc(Cl)cc2Cl)CC1      0   

   task2  task3  task4  task5  task6  task7  task8  task9  task10  task11  
0      0      0      0      0      0      0      0      0       0       0  
1      0      0      0      0      0      0      0     -1      -1       0  
2      0      0      0      0      0      0     -1      0       0       0  
3      0      0      0      0      0      0      0      1       1       1  
4      0      0      0      0      0      0      0      0       1       0  


In [20]:
print(df_test.head())

   Unnamed: 0                                             smiles
0           0                                    COc1cccc(C=O)c1
1           1                                    N#CC(O)c1ccccc1
2           2    Cl.Cl.OCCN1CCN(CCC=C2c3ccccc3Sc3ccc(Cl)cc32)CC1
3           3  CCN1CCN(c2ccc(C(C)(C)N=C(O)c3cc4[nH]c5ccccc5c(...
4           4                                 CCCc1ccc(O)c(OC)c1


In [21]:
print(df.shape)

(12000, 13)


In [22]:
print(df_test.shape)

(5896, 2)


In [23]:
df = df[(df == 0).sum(1) < 11] #remove the rows where all bioassay = 0 (unknown)

y_cols = df.columns[df.columns.str.contains(pat = 'task')] 
y = df[y_cols]

df.drop(y_cols, inplace=True, axis=1)
del df['Unnamed: 0']

# solve issue with 
y = y.replace(0, 0.5)
y = y.replace(-1, 0)

In [24]:
print(df)
print("\n")
print(y)

                                                  smiles
1                         CCOP(=S)(OCC)Oc1ccc(S(C)=O)cc1
2      CN(c1ccccc1CN=c1[nH]c(Nc2ccc3c(c2)CC(=O)N3)ncc...
3      CC#CC1(O)CCC2C3CCC4=CC(=O)CCC4=C3C(c3ccc(N(C)C...
4                      O=C(O)C1(C(O)=Nc2ccc(Cl)cc2Cl)CC1
5               CCN1C(=CC=Cc2sc3ccccc3[n+]2CC)Sc2ccccc21
...                                                  ...
11992                                      COS(=O)(=O)OC
11994      CCCCNc1cc(C(=O)[O-])cc(S(N)(=O)=O)c1Oc1ccccc1
11995  C[C@]12CC(=O)C3C(CCC4=CC(=O)CC[C@@]43C)C1CC[C@...
11997               OC[C@H]1NC[C@@H](O)[C@@H](O)[C@@H]1O
11998                 CN(C)CCCN1c2ccccc2C(C)(C)c2ccccc21

[11531 rows x 1 columns]


       task1  task2  task3  task4  task5  task6  task7  task8  task9  task10  \
1        0.5    0.5    0.5    0.5    0.5    0.5    0.5    0.5    0.0     0.0   
2        0.5    0.5    0.5    0.5    0.5    0.5    0.5    0.0    0.5     0.5   
3        0.5    0.5    0.5    0.5    0.5    0.5 

In [25]:
smiles = df["smiles"].to_list()

##### todo: n_atoms, n_bonds, bonds_type, one_hot_encode bonds available, one_hot_encode atoms available, molecule activity


df['n_atoms'] = [Chem.MolFromSmiles(i).GetNumAtoms() for i in smiles]
#df['activity'] = [Chem.MolFromSmiles(i).GetDoubleProp('Activity') for i in smiles]
#bonds = [ for bond in [Chem.MolFromSmiles(mol).GetBonds() for mol in smiles]] #(i.e. 1.0 for SINGLE, 1.5 for AROMATIC, 2.0 for DOUBLE)


In [26]:
d = Chem.MolFromSmiles(smiles[0])
print("Number of Atoms: ", d.GetNumAtoms())
print(Chem.MolToMolBlock(d, includeStereo=False)) #decomposes fragments

Number of Atoms:  18

     RDKit          

 18 18  0  0  0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 P   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 S   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.

In [27]:
#Define Descriptors
descriptor_list = Chem.Descriptors._descList
names = [desc[0] for desc in descriptor_list] #  select all available descriptors
MolDescCalc = MoleculeDescriptors.MolecularDescriptorCalculator(names)

In [28]:
print("# of columns descriptor df: ", len(MolDescCalc.CalcDescriptors(d)))

# of columns descriptor df:  208


In [72]:
# Build Descriptor Dataframe
mutag = [Chem.MolFromSmiles(i) for i in smiles[:5000]]
desc_mtx = np.zeros((len(mutag), len(names))) * np.nan
compounds = [''] * len(mutag)

# Calculate descriptors
for i, mol in enumerate(mutag): 
    if mol is not None:
        #compounds[i] = mol.GetProp('_Name')
        desc_mtx[i] = MolDescCalc.CalcDescriptors(mol)
        
df_des = pd.DataFrame(desc_mtx, columns=names)


In [73]:
df_des

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,11.254174,-2.722597,11.254174,0.438828,0.724283,308.361,291.225,308.030588,102.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.616488,-4.718719,13.616488,0.009742,0.474600,506.510,485.342,506.134794,184.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.169555,-0.913604,12.169555,0.220906,0.639526,429.604,394.324,429.266779,168.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.959321,-1.219281,10.959321,0.268740,0.654725,274.103,265.031,272.995949,90.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.378112,0.987633,2.378112,0.987633,0.547912,365.547,344.379,365.114067,126.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,10.825046,-0.103056,10.825046,0.001157,0.564836,144.214,128.086,144.115030,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,12.677643,-0.985038,12.677643,0.084820,0.382142,393.509,366.293,393.172227,148.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,14.422140,-0.369245,14.422140,0.019626,0.750508,387.886,364.702,387.151368,142.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,9.422891,-1.635645,9.422891,0.304583,0.805219,240.517,232.453,238.967147,72.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
#Linear Regression
import statsmodels.api as sm
rdkit_desc = rdkit_desc.fillna(0)

from sklearn import preprocessing

# x = rdkit_desc.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# rdkit_desc = pd.DataFrame(x_scaled)


df_des=(df_des-df_des.min())/(df_des.max()-df_des.min())
df_des = df_des.fillna(0)

cols = ["NumRadicalElectrons", "BertzCT", "NumAliphaticHeterocycles", "NumAliphaticRings", "NumAromaticCarbocycles", "NumAromaticRings", "NumSaturatedRings", "fr_C_O", 
        "fr_benzene", "fr_dihydropyridine", "fr_guanido", "fr_pyridine", "fr_urea", "qed"]

#df_des = df_des[cols]
x = sm.add_constant(df_des)


mod = sm.OLS(y["task8"][:5000].to_list(), x)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     8.006
Date:                Sat, 03 Apr 2021   Prob (F-statistic):          1.56e-175
Time:                        11:25:01   Log-Likelihood:                 750.71
No. Observations:                5000   AIC:                            -1113.
Df Residuals:                    4806   BIC:                             150.9
Df Model:                         193                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        5.8563      5.734      1.021      0.307      -5.384      17.097
MaxEStateIndex              -0.0322      0.020     -1.623      0.105      -0.071       0.007
MinEStateIndex               0.0836      0.063      1.327      0.185      -0.040       0.207
MaxAbsEStateIndex           -0.0322      0.020     -1.623      0.105      -0.071       0.007
MinAbsEStateIndex            0.1454      0.116      1.249      0.212      -0.083       0.374
qed                          0.0514      0.022      2.360      0.018       0.009       0.094
MolWt                     -136.2146    264.748     -0.515      0.607    -655.242     382.813
HeavyAtomMolWt               5.7894    126.175      0.046      0.963    -241.572     253.151
ExactMolWt                 131.9169    193.911      0.680      0.496    -248.237     512.070
NumValenceElectrons         21.7846     30.925      0.704      0.481     -38.843      82.412
NumRadicalElectrons          0.6433      0.283      2.270      0.023       0.088       1.199
MaxPartialCharge            -0.1246      0.290     -0.429      0.668      -0.693       0.444
MinPartialCharge             0.5091      0.226      2.249      0.025       0.065       0.953
MaxAbsPartialCharge          0.0406      0.201      0.202      0.840      -0.354       0.436
MinAbsPartialCharge          0.0540      0.153      0.353      0.724      -0.246       0.354
FpDensityMorgan1             0.0161      0.130      0.124      0.902      -0.239       0.271
FpDensityMorgan2             0.1347      0.239      0.563      0.574      -0.335       0.604
FpDensityMorgan3            -0.1856      0.150     -1.241      0.215      -0.479       0.108
BCUT2D_MWHI                 -0.0558      0.068     -0.825      0.409      -0.188       0.077
BCUT2D_MWLOW                 0.0726      0.262      0.277      0.782      -0.441       0.586
BCUT2D_CHGHI                 0.0054      0.238      0.023      0.982      -0.462       0.473
BCUT2D_CHGLO                 0.1181      0.149      0.793      0.428      -0.174       0.410
BCUT2D_LOGPHI                0.0626      0.240      0.261      0.794      -0.407       0.532
BCUT2D_LOGPLOW               0.0052      0.105      0.050      0.960      -0.201       0.212
BCUT2D_MRHI                 -0.0372      0.115     -0.323      0.747      -0.263       0.189
BCUT2D_MRLOW                -0.0482      0.068     -0.707      0.480      -0.182       0.086
BalabanJ                     0.2245      0.051      4.420      0.000       0.125       0.324
BertzCT                      4.8412      1.566      3.091      0.002       1.771       7.912
Chi0                        64.5443     26.194      2.464      0.014      13.192     115.896
Chi0n                       -8.1778     21.224     -0.385      0.700     -49.786      33.430
Chi0v         

In [94]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

input_size = len(df_des.columns)
output_size = 11
epochs = 10

# Create 3 layers
layer1 = layers.Dense(input_size, activation="sigmoid", name="layer1")
layer2 = layers.Dense(50, activation="tanh", name="layer2")
layer3 = layers.Dense(output_size, activation="softmax" name="layer3")

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy())

# Call layers on a test input

train_x = df_des.values
y = y.values
xx
history = model.fit(trainX, trainy, epochs=10, verbose=1)

SyntaxError: invalid syntax (<ipython-input-94-549dfd044ac8>, line 12)

In [91]:
cols = list(range(1,119))

In [92]:
df_ = pd.DataFrame()
df_[cols] = 0

In [93]:
df_

for atom_idx in atom_list:
    df.loc[df[atom_idx] == atom_idx, df[atom_idx]] += 1


,1,2,3,4,5,6,7,8,9,10,...,109,110,111,112,113,114,115,116,117,118
